# Examples of the same mission at different UI levels

Here we present a sort of [Rosetta Stone](https://en.wikipedia.org/wiki/Rosetta_Stone) for the same mission at different UI levels.
This is to show the code used to make and solve similar mission optimization problems at different UI levels.

In each of these cases we'll set the maximum iterations to 0 to save execution time.
If you wanted to actually solve these problems, you'd need to set the maximum iterations to a larger number, like 50.

```{note}
Because we're setting the max iter to 0, we see an "Optimization FAILED" output, but do not be concerned, this is expected as we're simply showing the different levels here, not actually solving the problems.
```

## Level 1

This is the most basic level of the UI.
There's a command-line interface (CLI) for Level 1, or you can use the Python API shown below.
If you want to make minor modifications to `phase_info`, you can do so here, but Level 1 largely assumes you're using the default setup.

In [ ]:
import aviary.api as av

prob = av.run_aviary('models/test_aircraft/aircraft_for_bench_FwFm.csv', av.default_height_energy_phase_info,
                    mission_method="FLOPS", mass_method="FLOPS", max_iter=0,
                    optimizer="SLSQP", make_plots=False)

## Level 2

This level grants more flexibility both in defining the `phase_info` object but also in calling the individual methods of `AviaryProblem` when setting up and running your model.
You can modify the methods you call, what they do, and what info they're given here.
This is much more verbose than Level 1.
In the absence of additional arguments to the methods, much of the default behavior here is the same as Level 1.

In [ ]:
from aviary.api import Dynamic
import aviary.api as av

phase_info = {
    'pre_mission': {
        'include_takeoff': True,
        'optimize_mass': True,
    },
    'climb': {
        'subsystem_options': {
            'core_aerodynamics': {'method': 'computed'}
        },
        'user_options': {
            'fix_initial': {Dynamic.Mission.MASS: False, Dynamic.Mission.RANGE: False},
            'fix_initial_time': True,
            'fix_duration': False,
            'num_segments': 6,
            'order': 3,
            'initial_altitude': (0., 'ft'),
            'initial_ref': (1., 's'),
            'initial_bounds': ((0., 500.), 's'),
            'duration_ref': (1452., 's'),
            'duration_bounds': ((726., 2904.), 's'),
            'final_altitude': (10668, 'm'),
            'input_initial': True,
            'no_descent': False,
            'initial_mach': 0.1,
            'final_mach': 0.79,
            'fix_range': False,
            'add_initial_mass_constraint': False,
        },
        'initial_guesses': {
            'times': ([2., 24.2], 'min'),
            'altitude': ([0., 35.e3], 'ft'),
            'velocity': ([220., 455.49], 'kn'),
            'mass': ([170.e3, 165.e3], 'lbm'),
            'range': ([0., 160.3], 'nmi'),
            'velocity_rate': ([0.25, 0.05], 'm/s**2'),
            'throttle': ([0.5, 0.5], 'unitless'),
        }
    },
    'cruise': {
        'subsystem_options': {
            'core_aerodynamics': {'method': 'computed'}
        },
        'user_options': {
            'fix_initial': False,
            'fix_final': False,
            'fix_duration': False,
            'num_segments': 1,
            'order': 3,
            'initial_ref': (1., 's'),
            'initial_bounds': ((500., 4000.), 's'),
            'duration_ref': (24370.8, 's'),
            'duration_bounds': ((726., 48741.6), 's'),
            'min_altitude': (10.668e3, 'm'),
            'max_altitude': (10.668e3, 'm'),
            'min_mach': 0.79,
            'max_mach': 0.79,
            'required_available_climb_rate': (1.524, 'm/s'),
            'mass_f_cruise': (1.e4, 'kg'),
            'range_f_cruise': (1.e6, 'm'),
        },
        'initial_guesses': {
            'times': ([26.2, 406.18], 'min'),
            'altitude': ([35.e3, 35.e3], 'ft'),
            'velocity': ([455.49, 455.49], 'kn'),
            'mass': ([165.e3, 140.e3], 'lbm'),
            'range': ([160.3, 3243.9], 'nmi'),
            'velocity_rate': ([0., 0.], 'm/s**2'),
            'throttle': ([0.95, 0.9], 'unitless'),
        }
    },
    'descent': {
        'subsystem_options': {
            'core_aerodynamics': {'method': 'computed'}
        },
        'user_options': {
            'fix_initial': False,
            'fix_range': True,
            'fix_duration': False,
            'num_segments': 5,
            'order': 3,
            'initial_ref': (1., 's'),
            'initial_bounds': ((10.e3, 30.e3), 's'),
            'duration_ref': (1754.4, 's'),
            'duration_bounds': ((726., 3508.8), 's'),
            'initial_altitude': (10.668e3, 'm'),
            'final_altitude': (10.668, 'm'),
            'initial_mach': 0.79,
            'final_mach': 0.3,
            'no_climb': False,
        },
        'initial_guesses': {
            'times': ([432.38, 29.24], 'min'),
            'altitude': ([35.e3, 35.], 'ft'),
            'velocity': ([455.49, 198.44], 'kn'),
            'mass': ([120.e3, 115.e3], 'lbm'),
            'range': ([3243.9, 3378.7], 'nmi'),
            'velocity_rate': ([-0.25, 0.0], 'm/s**2'),
            'throttle': ([0., 0.], 'unitless'),
        }
    },
    'post_mission': {
        'include_landing': True,
    },
}

csv_path = "models/test_aircraft/aircraft_for_bench_FwFm.csv"

prob = av.AviaryProblem(phase_info, mission_method="FLOPS", mass_method="FLOPS")

# Load aircraft and options data from user
# Allow for user overrides here
prob.load_inputs(csv_path)

# Have checks for clashing user inputs
prob.check_inputs()

prob.add_pre_mission_systems()

prob.add_phases()

prob.add_post_mission_systems()

# Link phases and variables
prob.link_phases()

prob.add_driver("SLSQP", max_iter=0)

prob.add_design_variables()

prob.add_objective(objective_type="mass", ref=-1e5)

prob.setup()

prob.set_initial_guesses()

prob.run_aviary_problem(record_filename='level2_example.db', suppress_solver_print=True, make_plots=False)

## Level 3

This level is the most flexible and the code will be the most verbose.
This directly calls the OpenMDAO and Dymos methods used to set up and run the model.
You have supreme control over every part of the model and mission definition, but that flexibility results in much more code.

In [ ]:
import dymos as dm
import openmdao.api as om
import scipy.constants as _units
from packaging import version

from aviary.api import HeightEnergyLandingPhaseBuilder as Landing
from aviary.api import HeightEnergyTakeoffPhaseBuilder as Takeoff
from aviary.api import HeightEnergyClimbPhaseBuilder as Climb
from aviary.api import HeightEnergyCruisePhaseBuilder as Cruise
from aviary.api import HeightEnergyDescentPhaseBuilder as Descent

from aviary.api import Aircraft, Dynamic, Mission

import aviary.api as av


use_new_dymos_syntax = version.parse(dm.__version__) > version.parse("1.8.0")

prob = om.Problem()

driver = prob.driver = om.ScipyOptimizeDriver()
opt_settings = prob.driver.opt_settings

driver.options['optimizer'] = 'SLSQP'
opt_settings['maxiter'] = 0
opt_settings['ftol'] = 5.0e-3
opt_settings['eps'] = 1e-2

##########################################
# Aircraft Input Variables and Options   #
##########################################

aviary_inputs = av.get_flops_inputs('LargeSingleAisle1FLOPS')
aviary_outputs = av.get_flops_outputs('LargeSingleAisle1FLOPS')

av.preprocess_options(aviary_inputs)

alt_airport = 0  # ft

alt_i_climb = 0*_units.foot  # m
alt_f_climb = 35000.0*_units.foot  # m
mass_i_climb = 180623*_units.lb  # kg
mass_f_climb = 176765*_units.lb  # kg
v_i_climb = 198.44*_units.knot  # m/s
v_f_climb = 455.49*_units.knot  # m/s
# initial mach set to lower value so it can intersect with takeoff end mach
# mach_i_climb = 0.3
mach_i_climb = 0.2
mach_f_climb = 0.79
range_i_climb = 0*_units.nautical_mile  # m
range_f_climb = 160.3*_units.nautical_mile  # m
t_i_climb = 2 * _units.minute  # sec
t_f_climb = 26.20*_units.minute  # sec
t_duration_climb = t_f_climb - t_i_climb

alt_i_cruise = 35000*_units.foot  # m
alt_f_cruise = 35000*_units.foot  # m
alt_min_cruise = 35000*_units.foot  # m
alt_max_cruise = 35000*_units.foot  # m
mass_i_cruise = 176765*_units.lb  # kg
mass_f_cruise = 143521*_units.lb  # kg
v_i_cruise = 455.49*_units.knot  # m/s
v_f_cruise = 455.49*_units.knot  # m/s
# mach_i_cruise = 0.79
# mach_f_cruise = 0.79
mach_min_cruise = 0.79
mach_max_cruise = 0.79
range_i_cruise = 160.3*_units.nautical_mile  # m
range_f_cruise = 3243.9*_units.nautical_mile  # m
t_i_cruise = 26.20*_units.minute  # sec
t_f_cruise = 432.38*_units.minute  # sec
t_duration_cruise = t_f_cruise - t_i_cruise

alt_i_descent = 35000*_units.foot
# final altitude set to 35 to ensure landing is feasible point
# alt_f_descent = 0*_units.foot
alt_f_descent = 35*_units.foot
v_i_descent = 455.49*_units.knot
v_f_descent = 198.44*_units.knot
mach_i_descent = 0.79
mach_f_descent = 0.3
mass_i_descent = 143521*_units.pound
mass_f_descent = 143035*_units.pound
range_i_descent = 3243.9*_units.nautical_mile
range_f_descent = 3378.7*_units.nautical_mile
t_i_descent = 432.38*_units.minute
t_f_descent = 461.62*_units.minute
t_duration_descent = t_f_descent - t_i_descent

##################
# Define Phases  #
##################

num_segments_climb = 6
num_segments_cruise = 1
num_segments_descent = 5

climb_seg_ends, _ = dm.utils.lgl.lgl(num_segments_climb + 1)
descent_seg_ends, _ = dm.utils.lgl.lgl(num_segments_descent + 1)

transcription_climb = dm.Radau(
    num_segments=num_segments_climb, order=3, compressed=True,
    segment_ends=climb_seg_ends)
transcription_cruise = dm.Radau(
    num_segments=num_segments_cruise, order=3, compressed=True)
transcription_descent = dm.Radau(
    num_segments=num_segments_descent, order=3, compressed=True,
    segment_ends=descent_seg_ends)

takeoff_options = Takeoff(
    airport_altitude=alt_airport,  # ft
    num_engines=aviary_inputs.get_val(Aircraft.Engine.NUM_ENGINES)
)

climb_options = Climb(
    'test_climb',
    user_options=av.AviaryValues({
        'initial_altitude': (alt_i_climb, 'm'),
        'final_altitude': (alt_f_climb, 'm'),
        'initial_mach': (mach_i_climb, 'unitless'),
        'final_mach': (mach_f_climb, 'unitless'),
        'fix_initial': (False, 'unitless'),
        'fix_range': (False, 'unitless'),
        'input_initial': (True, 'unitless'),
    }),
    core_subsystems=av.default_mission_subsystems,
    subsystem_options={'core_aerodynamics': {'method': 'computed'}},
    transcription=transcription_climb,
)

cruise_options = Cruise(
    'test_cruise',
    user_options=av.AviaryValues({
        'min_altitude': (alt_min_cruise, 'm'),
        'max_altitude': (alt_max_cruise, 'm'),
        'min_mach': (mach_min_cruise, 'unitless'),
        'max_mach': (mach_max_cruise, 'unitless'),
        'required_available_climb_rate': (300, 'ft/min'),
        'fix_initial': (False, 'unitless'),
        'fix_final': (False, 'unitless'),
    }),
    core_subsystems=av.default_mission_subsystems,
    subsystem_options={'core_aerodynamics': {'method': 'computed'}},
    transcription=transcription_cruise,
)

descent_options = Descent(
    'test_descent',
    user_options=av.AviaryValues({
        'final_altitude': (alt_f_descent, 'm'),
        'initial_altitude': (alt_i_descent, 'm'),
        'initial_mach': (mach_i_descent, 'unitless'),
        'final_mach': (mach_f_descent, 'unitless'),
        'fix_initial': (False, 'unitless'),
        'fix_range': (True, 'unitless'),
    }),
    core_subsystems=av.default_mission_subsystems,
    subsystem_options={'core_aerodynamics': {'method': 'computed'}},
    transcription=transcription_descent,
)

landing_options = Landing(
    ref_wing_area=aviary_inputs.get_val(Aircraft.Wing.AREA, 'ft**2'),
    Cl_max_ldg=aviary_inputs.get_val(Mission.Landing.LIFT_COEFFICIENT_MAX)
)

# Upstream static analysis for aero
prob.model.add_subsystem(
    'pre_mission',
    av.CorePreMission(aviary_options=aviary_inputs,
                    subsystems=av.default_premission_subsystems),
    promotes_inputs=['aircraft:*', 'mission:*'],
    promotes_outputs=['aircraft:*', 'mission:*'])

# directly connect phases (strong_couple = True), or use linkage constraints (weak
# coupling / strong_couple=False)
strong_couple = False

takeoff = takeoff_options.build_phase(False)

climb = climb_options.build_phase(aviary_options=aviary_inputs)

cruise = cruise_options.build_phase(aviary_options=aviary_inputs)

descent = descent_options.build_phase(aviary_options=aviary_inputs)

landing = landing_options.build_phase(False)

prob.model.add_subsystem(
    'takeoff', takeoff, promotes_inputs=['aircraft:*', 'mission:*'],
    promotes_outputs=['mission:*'])

traj = prob.model.add_subsystem('traj', dm.Trajectory())

# if fix_initial is false, can we always set input_initial to be true for
# necessary states, and then ignore if we use a linkage?
climb.set_time_options(
    fix_initial=True, fix_duration=False, units='s',
    duration_bounds=(1, t_duration_climb*2), duration_ref=t_duration_climb)
cruise.set_time_options(
    fix_initial=False, fix_duration=False, units='s',
    duration_bounds=(1, t_duration_cruise*2), duration_ref=t_duration_cruise)
descent.set_time_options(
    fix_initial=False, fix_duration=False, units='s',
    duration_bounds=(1, t_duration_descent*2), duration_ref=t_duration_descent)

traj.add_phase('climb', climb)

traj.add_phase('cruise', cruise)

traj.add_phase('descent', descent)

if use_new_dymos_syntax:
    climb.timeseries_options['use_prefix'] = True
    cruise.timeseries_options['use_prefix'] = True
    descent.timeseries_options['use_prefix'] = True

prob.model.add_subsystem(
    'landing', landing, promotes_inputs=['aircraft:*', 'mission:*'],
    promotes_outputs=['mission:*'])

traj.link_phases(["climb", "cruise"], ["time", Dynamic.Mission.ALTITUDE,
                    Dynamic.Mission.VELOCITY, Dynamic.Mission.MASS, Dynamic.Mission.RANGE], connected=strong_couple)
traj.link_phases(["cruise", "descent"], ["time", Dynamic.Mission.ALTITUDE,
                    Dynamic.Mission.VELOCITY, Dynamic.Mission.MASS, Dynamic.Mission.RANGE], connected=strong_couple)

traj = av.setup_trajectory_params(prob.model, traj, aviary_inputs)

##################################
# Connect in Takeoff and Landing #
##################################

prob.model.add_subsystem(
    "takeoff_constraints",
    om.ExecComp(
        [
            "takeoff_mass_con=takeoff_mass-climb_start_mass",
            "takeoff_range_con=takeoff_range-climb_start_range",
            "takeoff_vel_con=takeoff_vel-climb_start_vel",
            "takeoff_alt_con=takeoff_alt-climb_start_alt"
        ],
        takeoff_mass_con={'units': 'lbm'}, takeoff_mass={'units': 'lbm'},
        climb_start_mass={'units': 'lbm'},
        takeoff_range_con={'units': 'ft'}, takeoff_range={'units': 'ft'},
        climb_start_range={'units': 'ft'},
        takeoff_vel_con={'units': 'm/s'}, takeoff_vel={'units': 'm/s'},
        climb_start_vel={'units': 'm/s'},
        takeoff_alt_con={'units': 'ft'}, takeoff_alt={'units': 'ft'},
        climb_start_alt={'units': 'ft'}
    ),
    promotes_inputs=[
        ("takeoff_mass", Mission.Takeoff.FINAL_MASS),
        ("takeoff_range", Mission.Takeoff.GROUND_DISTANCE),
        ("takeoff_vel", Mission.Takeoff.FINAL_VELOCITY),
        ("takeoff_alt", Mission.Takeoff.FINAL_ALTITUDE),
    ],
    promotes_outputs=["takeoff_mass_con", "takeoff_range_con",
                        "takeoff_vel_con", "takeoff_alt_con"],
)

prob.model.connect('traj.climb.states:mass',
                    'takeoff_constraints.climb_start_mass', src_indices=[0])
prob.model.connect('traj.climb.states:range',
                    'takeoff_constraints.climb_start_range', src_indices=[0])
prob.model.connect('traj.climb.states:velocity',
                    'takeoff_constraints.climb_start_vel', src_indices=[0])
prob.model.connect('traj.climb.states:altitude',
                    'takeoff_constraints.climb_start_alt', src_indices=[0])

prob.model.connect(Mission.Takeoff.FINAL_MASS,
                    'traj.climb.initial_states:mass')
prob.model.connect(Mission.Takeoff.GROUND_DISTANCE,
                    'traj.climb.initial_states:range')
prob.model.connect(Mission.Takeoff.FINAL_VELOCITY,
                    'traj.climb.initial_states:velocity')
prob.model.connect(Mission.Takeoff.FINAL_ALTITUDE,
                    'traj.climb.initial_states:altitude')

prob.model.connect('traj.descent.states:mass',
                    Mission.Landing.TOUCHDOWN_MASS, src_indices=[-1])
prob.model.connect('traj.descent.states:altitude', Mission.Landing.INITIAL_ALTITUDE,
                    src_indices=[-1])

##########################
# Add Objective Function #
##########################

# This is an example of a overall mission objective
# create a compound objective that minimizes climb time and maximizes final mass
# we are maxing final mass b/c we don't have an independent value for fuel_mass yet
# we are going to normalize these (makign each of the sub-objectives approx = 1 )
prob.model.add_subsystem(
    "regularization",
    om.ExecComp(
        # TODO: change the scaling on climb_duration
        "reg_objective = - descent_mass_final/60000",
        reg_objective=0.0,
        descent_mass_final={"units": "kg", "shape": 1},
    ),
    promotes_outputs=['reg_objective']
)
# connect the final mass from cruise into the objective
prob.model.connect("traj.descent.states:mass",
                    "regularization.descent_mass_final", src_indices=[-1])

prob.model.add_objective('reg_objective', ref=1)

# Set initial default values for all aircraft variables.
av.set_aviary_initial_values(prob.model, aviary_inputs)

# TODO: Why is this in outputs and not inputs?
key = Aircraft.Engine.THRUST_REVERSERS_MASS
val, units = aviary_outputs.get_item(key)
prob.model.set_input_defaults(key, val, units)

prob.model.add_subsystem(
    'input_sink',
    av.VariablesIn(aviary_options=aviary_inputs),
    promotes_inputs=['*'],
    promotes_outputs=['*']
)

prob.setup()

###########################################
# Intial Settings for States and Controls #
###########################################

prob.set_val('traj.climb.t_initial', t_i_climb, units='s')
prob.set_val('traj.climb.t_duration', t_duration_climb, units='s')

prob.set_val('traj.climb.states:altitude', climb.interp(
    Dynamic.Mission.ALTITUDE, ys=[alt_i_climb, alt_f_climb]), units='m')
# prob.set_val(
#     'traj.climb.states:velocity', climb.interp(Dynamic.Mission.VELOCITY, ys=[170, v_f_climb]),
#     units='m/s')
prob.set_val('traj.climb.states:velocity', climb.interp(
    Dynamic.Mission.VELOCITY, ys=[v_i_climb, v_f_climb]), units='m/s')
prob.set_val('traj.climb.states:mass', climb.interp(
    Dynamic.Mission.MASS, ys=[mass_i_climb, mass_f_climb]), units='kg')
prob.set_val('traj.climb.states:range', climb.interp(
    Dynamic.Mission.RANGE, ys=[range_i_climb, range_f_climb]), units='m')  # nmi

prob.set_val('traj.climb.controls:velocity_rate',
                climb.interp(Dynamic.Mission.VELOCITY_RATE, ys=[0.25, 0.0]),
                units='m/s**2')
prob.set_val('traj.climb.controls:throttle',
                climb.interp(Dynamic.Mission.THROTTLE, ys=[0.5, 0.5]),
                units='unitless')

prob.set_val('traj.cruise.t_initial', t_i_cruise, units='s')
prob.set_val('traj.cruise.t_duration', t_duration_cruise, units='s')

prob.set_val('traj.cruise.states:altitude', cruise.interp(
    Dynamic.Mission.ALTITUDE, ys=[alt_i_cruise, alt_f_cruise]), units='m')
prob.set_val('traj.cruise.states:velocity', cruise.interp(
    Dynamic.Mission.VELOCITY, ys=[v_i_cruise, v_f_cruise]), units='m/s')
prob.set_val('traj.cruise.states:mass', cruise.interp(
    Dynamic.Mission.MASS, ys=[mass_i_cruise, mass_f_cruise]), units='kg')
prob.set_val('traj.cruise.states:range', cruise.interp(
    Dynamic.Mission.RANGE, ys=[range_i_cruise, range_f_cruise]), units='m')  # nmi

prob.set_val('traj.cruise.controls:velocity_rate',
                cruise.interp(Dynamic.Mission.VELOCITY_RATE, ys=[0.0, 0.0]),
                units='m/s**2')
prob.set_val('traj.cruise.controls:throttle',
                cruise.interp(Dynamic.Mission.THROTTLE, ys=[0.5, 0.5]),
                units='unitless')

prob.set_val('traj.descent.t_initial', t_i_descent, units='s')
prob.set_val('traj.descent.t_duration', t_duration_descent, units='s')

prob.set_val('traj.descent.states:altitude', descent.interp(
    Dynamic.Mission.ALTITUDE, ys=[alt_i_descent, alt_f_descent]), units='m')
prob.set_val('traj.descent.states:velocity', descent.interp(
    Dynamic.Mission.VELOCITY, ys=[v_i_descent, v_f_descent]), units='m/s')
prob.set_val('traj.descent.states:mass', descent.interp(
    Dynamic.Mission.MASS, ys=[mass_i_descent, mass_f_descent]), units='kg')
prob.set_val('traj.descent.states:range', descent.interp(
    Dynamic.Mission.RANGE, ys=[range_i_descent, range_f_descent]), units='m')

prob.set_val('traj.descent.controls:velocity_rate',
                descent.interp(Dynamic.Mission.VELOCITY_RATE, ys=[-0.25, 0.0]),
                units='m/s**2')
prob.set_val('traj.descent.controls:throttle',
                descent.interp(Dynamic.Mission.THROTTLE, ys=[0.0, 0.0]),
                units='unitless')

# Turn off solver printing so that the SNOPT output is readable.
prob.set_solver_print(level=0)

dm.run_problem(prob, simulate=False, make_plots=False, simulate_kwargs={
                'times_per_seg': 100, 'atol': 1e-9, 'rtol': 1e-9},
                solution_record_file='large_single_aisle_1_solution.db')

